# LINE Messaging API Learning Notebook

This notebook covers the fundamental aspects of the LINE Messaging API based on the official documentation. We will explore:
- **Message**: Sending messages (Reply, Push)
- **Insights**: Retrieving statistical data
- **Managing Audience**: Creating and managing audiences for narrowcasting
- **User**: Retrieving user profile information

---

## 🛠 Setup & Authentication

To use the LINE Messaging API, you need a **Channel Access Token**. You can get this from the [LINE Developers Console](https://developers.line.biz/console/).

In [ ]:
import requests
import json
import os
from datetime import datetime

# Constants - Replace with your own credentials
CHANNEL_ACCESS_TOKEN = 'YOUR_CHANNEL_ACCESS_TOKEN'
USER_ID = 'YOUR_USER_ID'  # Your own user ID for testing push messages

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {CHANNEL_ACCESS_TOKEN}'
}

---
## 💬 1. Message API

The Message API allows you to send various types of messages to users. 

### 1.1 Sending a Push Message
Push messages can be sent at any time to users who have added your bot as a friend.

- **Endpoint**: `POST https://api.line.me/v2/bot/message/push`

In [ ]:
def send_push_message(user_id, text):
    url = 'https://api.line.me/v2/bot/message/push'
    payload = {
        'to': user_id,
        'messages': [
            {
                'type': 'text',
                'text': text
            }
        ]
    }
    
    response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
    return response.status_code, response.json() if response.status_code != 200 else "Success"

# Example usage (Uncomment to run if you have set credentials)
# status, result = send_push_message(USER_ID, "Hello from Jupyter!")
# print(f"Status: {status}")
# print(f"Result: {result}")

### 1.2 Sending a Reply Message
Reply messages are sent in response to a webhook event (like a user sending a message). You must use a `replyToken` provided in the webhook payload.

- **Endpoint**: `POST https://api.line.me/v2/bot/message/reply`

In [ ]:
def send_reply_message(reply_token, text):
    url = 'https://api.line.me/v2/bot/message/reply'
    payload = {
        'replyToken': reply_token,
        'messages': [
            {
                'type': 'text',
                'text': text
            }
        ]
    }
    
    response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
    return response.status_code, response.json() if response.status_code != 200 else "Success"

print("Note: Reply tokens can only be used once and expire quickly!")

---
## 📊 2. Insights API

Retrieves statistics such as message delivery counts, friend counts, etc.

### 2.1 Get Number of Message Deliveries
Check how many messages were sent on a specific date.

- **Endpoint**: `GET https://api.line.me/v2/bot/insight/message/delivery?date={date}`

In [ ]:
def get_message_deliveries(date_str):
    # date_str format: yyyyMMdd (e.g., 20231027)
    url = f'https://api.line.me/v2/bot/insight/message/delivery?date={date_str}'
    
    response = requests.get(url, headers=HEADERS)
    return response.json()

# Example: yesterday = datetime.now().strftime('%Y%m%d')
# print(get_message_deliveries('20231026'))

### 2.2 Get Friend Demographics
Get information about your friends' gender, age group, area, etc.

- **Endpoint**: `GET https://api.line.me/v2/bot/insight/demographic`

In [ ]:
def get_demographics():
    url = 'https://api.line.me/v2/bot/insight/demographic'
    
    response = requests.get(url, headers=HEADERS)
    return response.json()

# print(get_demographics())

---
## 👥 3. Managing Audience API

Create and manage audiences for **Narrowcast** messages (targeting specific users).

### 3.1 Create Audience for Uploading User IDs
You can create an audience by uploading a list of user IDs.

- **Endpoint**: `POST https://api.line.me/v2/bot/audienceGroup/upload`

In [ ]:
def create_audience(description, user_ids):
    url = 'https://api.line.me/v2/bot/audienceGroup/upload'
    payload = {
        'description': description,
        'audiences': [{'id': uid} for uid in user_ids]
    }
    
    response = requests.post(url, headers=HEADERS, data=json.dumps(payload))
    return response.json()

# Example list of IDs
# my_users = ['U123...', 'U456...']
# print(create_audience('My Test Audience', my_users))

### 3.2 Get Audience Data
Check the status and size of a specific audience.

- **Endpoint**: `GET https://api.line.me/v2/bot/audienceGroup/{audienceGroupId}`

In [ ]:
def get_audience_info(audience_id):
    url = f'https://api.line.me/v2/bot/audienceGroup/{audience_id}'
    
    response = requests.get(url, headers=HEADERS)
    return response.json()

# print(get_audience_info('1234567890'))

---
## 👤 4. User API

Get profile information of users who have added your bot as a friend.

### 4.1 Get User Profile
Retrieve the display name, profile picture, and status message of a user.

- **Endpoint**: `GET https://api.line.me/v2/bot/profile/{userId}`

In [ ]:
def get_user_profile(user_id):
    url = f'https://api.line.me/v2/bot/profile/{user_id}'
    
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        profile = response.json()
        print(f"Display Name: {profile.get('displayName')}")
        print(f"User ID: {profile.get('userId')}")
        print(f"Picture URL: {profile.get('pictureUrl')}")
        print(f"Status Message: {profile.get('statusMessage')}")
        return profile
    else:
        return response.json()

# get_user_profile(USER_ID)

---
## 💡 Summary

In this notebook, we learned how to:
1. **Send Messages**: Push and Reply messages.
2. **Get Insights**: Monitor delivery stats and friend demographics.
3. **Manage Audiences**: Create groups for targeted messaging.
4. **Fetch User Profiles**: Get details about users.

For full details, refer to the local `api_doc.md` or the [Official LINE Messaging API Documentation](https://developers.line.biz/en/reference/messaging-api/).